# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [11]:
print(get_links_user_prompt("https://hayatusanusi.xyz"))


Here is the list of links on the website https://hayatusanusi.xyz -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/blog
mailto:hayatusanusi.dev@gmail.com
https://github.com/hamicch?ref=hayatusanusi.xyz
https://linkedin.com/in/sanusihayatu?ref=hayatusanusi.xyz
https://x-hr.co/?ref=hayatusanusi.xyz
https://tamara.co/?ref=hayatusanusi.xyz
https://tiqwa.com/?ref=hayatusanusi.xyz
https://chats.cash/?ref=hayatusanusi.xyz
https://cngn.co/?ref=hayatusanusi.xyz
/blog


In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links


In [ ]:
select_relevant_links("https://edwarddonner.com")

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://hayatusanusi.xyz")

Selecting relevant links for https://hayatusanusi.xyz by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'blog', 'url': 'https://hayatusanusi.xyz/blog'},
  {'type': 'GitHub profile',
   'url': 'https://github.com/hamicch?ref=hayatusanusi.xyz'},
  {'type': 'LinkedIn profile',
   'url': 'https://linkedin.com/in/sanusihayatu?ref=hayatusanusi.xyz'},
  {'type': 'partner site', 'url': 'https://x-hr.co/?ref=hayatusanusi.xyz'},
  {'type': 'partner site', 'url': 'https://tamara.co/?ref=hayatusanusi.xyz'},
  {'type': 'partner site', 'url': 'https://tiqwa.com/?ref=hayatusanusi.xyz'},
  {'type': 'partner site', 'url': 'https://chats.cash/?ref=hayatusanusi.xyz'},
  {'type': 'partner site', 'url': 'https://cngn.co/?ref=hayatusanusi.xyz'}]}

In [13]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'github', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'forum/community', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-397B-A17B
Updated
2 days ago
•
218k
•
865
nvidia/personaplex-7b-v1
Updated
7 days ago
•
540k
•
2.13k
Nanbeige/Nanbeige4.1-3B
Updated
about 12 hours ago
•
154k
•
691
zai-org/GLM-5
Updated
9 days ago
•
178k
•
1.42k
MiniMaxAI/MiniMax-M2.5
Updated
6 days ago
•
191k
•
847
Browse 2M+ models
Spaces
Running
Featured
4.79k
Wan2.2 Animate
👁
4.79k
Wan2.2 Animate
Running
on
Zero
MCP
855
Wan2.2 14B Preview
🐌
855
gen

In [ ]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-397B-A17B\nUpdated\n2 days ago\n•\n218k\n•\n865\nnvidia/personaplex-7b-v1\nUpdated\n7 days ago\n•\n540k\n•\n2.13k\nNanbeige/Nanbeige4.1-3B\nUpdated\nabout 12 hours ago\n•\n154k\n•\n691\nzai-org/GLM-5\nUpdated\n9 days ago\n•\n178k\n•\n1.42k\nMiniMaxAI/Mini

In [21]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano


KeyboardInterrupt: 

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face  
*The AI Community Building the Future*

---

## About Hugging Face

Hugging Face is a pioneering platform and community at the forefront of machine learning (ML) and artificial intelligence (AI) innovation. Dedicated to accelerating the future of AI, Hugging Face provides an open and collaborative environment where developers, researchers, and enterprises can create, share, and advance ML models, datasets, and applications.

With a thriving ecosystem, Hugging Face hosts **2+ million ML models**, **500k+ datasets**, and **1 million+ applications**, making it the go-to hub for AI practitioners worldwide.

---

## Key Features

### Collaborative Platform  
- Host and collaborate on **unlimited public and private ML models, datasets, and AI applications**.  
- Engage with an active global community contributing to the evolution of AI technologies.

### Extensive & Diverse AI Resources  
- Explore models spanning multiple modalities including **text, image, video, audio, and 3D**.  
- Access and contribute to trending state-of-the-art models such as Qwen3.5, personaplex, and many more.  
- Millions of datasets available for research and development.

### Custom AI Applications & Spaces  
- Launch and demo AI-powered applications right on the platform with **Spaces** — interactive apps for various AI use-cases like image generation, text-to-speech, and video creation.  
- Experiment with the latest in AI like HuggingChat Omni for conversational AI experiences.

### Open Source & Enterprise Ready  
- Benefit from the robust Hugging Face open-source ML stack to accelerate development and deployment.  
- Enterprise customers can leverage tailored solutions and professional support.

---

## Company Culture

Hugging Face embodies a spirit of **openness**, **collaboration**, and **innovation**. The company nurtures a passionate AI community, believing that **open data and shared resources are the keys to advancing AI for all**. Contributors from academia, research labs, startups, and industry converge here to push the boundaries of what's possible.

The culture encourages building a **strong, diverse, and engaged community** where knowledge sharing and creative experimentation thrive alongside production-grade ML tools.

---

## Customers & Community

Hugging Face serves a vibrant mix of users:  
- AI researchers and data scientists accelerating model development.  
- Developers building the next generation of intelligent applications.  
- Enterprises deploying AI solutions at scale across industries such as technology, healthcare, and finance.  
- Educators and students learning and teaching practical AI skills.

Regular community events, forums, and feedback loops keep users connected and growing together.

---

## Careers at Hugging Face

If you are passionate about advancing AI and shaping the future, Hugging Face offers exciting career opportunities in:  
- Machine Learning Research  
- Software Development  
- Community Management  
- Data Engineering  
- Product and Design  

Working at Hugging Face means joining a team dedicated to open innovation, diversity, and impactful AI work.

**Explore openings:** Visit the Hugging Face website for current job listings.

---

## Get Started

- **Explore AI Apps and Models:** Access millions of models and applications.  
- **Build and Share:** Host your projects and build your professional ML profile.  
- **Join the Community:** Connect with innovators pushing AI forward.  
- **Enterprise Solutions:** Transform your business with specialized AI tools.

**Sign Up or Log In:** experience the future of AI collaboration today at [huggingface.co](https://huggingface.co)

---

*Hugging Face — Empowering the AI community to build a better tomorrow.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>